# CycleGAN for Faces Dataset

**Objective:** Implement CycleGAN for faces dataset.

Dataset Location:  https://susanqq.github.io/UTKFace/

Original CycleGAN Implementation in PyTorch is available at https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

## Import Dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp '/content/drive/My Drive/App/CycleGAN/Faces/low_res/young.128.npz' /content
!cp '/content/drive/My Drive/App/CycleGAN/Faces/low_res/old.128.npz'   /content

## Imports

In [0]:
import numpy as np
import os
from PIL import Image

import matplotlib.pyplot as plt

%matplotlib inline
import datetime

## Load the dataset

In [4]:
# load the compressed data
young = np.load('/content/young.128.npz')
young_images = young['arr_0']
print(young_images.shape)

(500, 128, 128, 3)


In [5]:
# load the old faces data
old = np.load('/content/old.128.npz')
old_images = old['arr_0']
print(old_images.shape)

(500, 128, 128, 3)


## Rescale the values to -1 to 1

In [0]:
def rescale_input(data_arr):
  res_arr = (data_arr - 127.5) / 127.5
  return res_arr

## Install external packages

In [7]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-64vivpo4
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-64vivpo4
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101066 sha256=60234cd731bcc41e81dd739ef40ab4daff9a7526b7d0a55b118546b3e32f4eb7
  Stored in directory: /tmp/pip-ephem-wheel-cache-dou_l2uz/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


## Build CycleGAN

In [8]:
# keras layers
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate, Lambda, add
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv2DTranspose
from keras.initializers import RandomNormal

# from keras_contrib
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization

from keras.models import Sequential, Model

from keras.optimizers import Adam

Using TensorFlow backend.


### Discriminator

For Discriminator:
*  Use PatchGAN - only penalizes the structure at the scale of patches.
* PatchGAN classifies the NxN patch is real or fake
*  They have fewer parameters than the full image discriminator
* PatchGAN are used in [Image to Image translation](https://arxiv.org/pdf/1611.07004.pdf)

In [0]:
# Discriminator layer has the following
#  * Conv2D - filter size: 4x4, strides:2
#  * LeakyReLU
#  * InstanceNormalization
#
def d_layer(layer_input, filters, f_size=4, normalization=True):
  d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
  d = LeakyReLU(alpha=0.2)(d)
  if normalization:
      d = InstanceNormalization()(d)
  return d

In [0]:
# build discriminator uses PatchGAN
# Uses the patch to classify the image is fake or real.
# PatchGAN uses 
#  * kernel size 4x4
#  * num filters double at each stage
def build_discriminator(image_shape, num_start_filters=64):
  img = Input(image_shape)
  
  d1 = d_layer(img, num_start_filters, normalization=False)
  d2 = d_layer(d1, num_start_filters*2)
  d3 = d_layer(d2, num_start_filters*4)
  d4 = d_layer(d3, num_start_filters*8)

  validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)
  
  model = Model(img, validity)
  
  # compile model
  model.compile(loss='mse', optimizer=Adam(lr=0.0002, beta_1=0.5), loss_weights=[0.5])
  
  return model

### Generator

Generator can be one of the following two things:

     * Encoder : Decoder combo (UNet - uses skip connections)
     or
     * Encoder : Transformer : Decoder (Uses Residual blocks)
     
 The Encoder shrinks the input image. Uses Conv layers (with strides:2).
 
 The Transformer uses residual blocks
 
 The Decoder expands the image with transpose Conv.
 
 Note: each layer will use LeakyReLU and InstanceNormalization

#### Resnet block

Original paper uses **reflection padding**. Let's use **same** padding for simplicity.

In [0]:
'''
def resnet_block(r_i, layer_output, ks=3, s=1):
    r = Lambda(lambda x: tf.pad(x, [[0,0],[1,1],[1,1],[0,0]],'REFLECT'))(r_i)
    #r = ReflectionPadding2D(padding=(1,1))(r_i)
    r = conv2d(r,layer_output,ks,s,padding= 'VALID')
    r = InstanceNormalization()(r)
    
    r = Lambda(lambda x: tf.pad(x, [[0,0],[1,1],[1,1],[0,0]],'REFLECT'))(r)
    #r = ReflectionPadding2D(padding=(1,1))(r)
    r = conv2d(r,layer_output,ks,s,padding= 'VALID')
    r = InstanceNormalization()(r)
    
    return add([r_i , r])
'''
  
def resnet_block(n_filters, input_layer):
  # first layer convolutional layer
	g = Conv2D(n_filters, (3,3), padding='same')(input_layer)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
  
	# second convolutional layer
	g = Conv2D(n_filters, (3,3), padding='same')(g)
	g = InstanceNormalization(axis=-1)(g)
  
	# concatenate merge channel-wise with input layer
	g = Concatenate()([g, input_layer])
	return g


In [0]:
# define  generator model
def build_generator(image_shape, n_resnet=9):
	 
	# image input
	in_image = Input(shape=image_shape)
  
  ## 
	# c7s1-64
	g = Conv2D(64, (7,7), padding='same')(in_image)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
  
  ## down sample
	# d128
	g = Conv2D(128, (3,3), strides=(2,2), padding='same')(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# d256
	g = Conv2D(256, (3,3), strides=(2,2), padding='same')(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
  
	# R256 - resnet blocks
	for _ in range(n_resnet):
		g = resnet_block(256, g)
    
  ## upsample
	# u128
	g = Conv2DTranspose(128, (3,3), strides=(2,2), padding='same')(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# u64
	g = Conv2DTranspose(64, (3,3), strides=(2,2), padding='same')(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# c7s1-3
	g = Conv2D(3, (7,7), padding='same')(g)
	g = InstanceNormalization(axis=-1)(g)
	out_image = Activation('tanh')(g)
  
	# define model
	model = Model(in_image, out_image)
	return model

### Build combined model

In [0]:
## build combined model generator1, discrimator1 and generator2
##
## 1. The cycle loss is given more weightage 10 time more than
##    the adversarial loss.
##
## 2. The identity loss is half the weightage of the cycle loss
##    so, it is 5

def build_combined_model(image_shape, g_model1, d_model, g_model2):
  # update the trainable flag
  g_model1.trainable = True
  d_model.trainable = False
  g_model2.trainable = False
  
  # discriminator elelemnt
  input_gen = Input(shape=image_shape)
  
  gen1_out = g_model1(input_gen)
  output_d = d_model(gen1_out)
  
  # identity element
  input_id = Input(shape=image_shape)
  output_id = g_model1(input_id)
  
  # forward cycle
  output_f = g_model2(gen1_out)
  
  # backward cycle
  gen2_out = g_model2(input_id)
  output_b = g_model1(gen2_out)
  
  #define combined model
  model = Model([input_gen, input_id], [output_d, output_id, output_f, output_b])
  
  #define the optimizer
  opt = Adam(lr=0.0002, beta_1=0.5)
  
  # compile model with weighting of least squares loss and L1 loss
  ## Cycle loss is 10 times more than adv. loss
  ## ident loss is half the cylce loss
  model.compile(loss=['mse', 'mae', 'mae', 'mae'], loss_weights=[1, 5, 10, 10], optimizer=opt)
  return model

## Training

In [0]:
# generate real samples
def gen_real_samples(dataset, n_samples, patch_size):
  # get random image
  idx = np.random.randint(0, dataset.shape[0], n_samples)
  
  X = dataset[idx]
  
  # generate labels as well.
  y = np.ones((n_samples, patch_size, patch_size, 1))
  return X, y

  

In [0]:
# generate fake images from generator
def gen_fake_samples(g_model, images, patch_size):
  # use model predictor
  fake_images = g_model.predict(images)
  
  # mark the labels as zero for fake.
  y = np.zeros((len(fake_images), patch_size, patch_size, 1))
  return fake_images, y

In [0]:
# save models to file
def save_models(epoch, directory, gen_AB, gen_BA):
  file_name1 = 'gen_modelAB_e%03d.hdf5' % (epoch)
  path1 = os.path.join(directory, file_name1)
  
  file_name2 = 'gen_modelBA_e%03d.hdf5' % (epoch)
  path2 = os.path.join(directory, file_name2)
  
  gen_AB.save(path1)
  gen_BA.save(path2)
  return

In [0]:
# sample images at regular intervals to see the progress
def sample_images(epoch, step, gen_model, trainX, prefix, directory, n_samples=5):
  # get the real samples
  X_in, _ = gen_real_samples(trainX, n_samples, 0)
  
  # get the generated / translated images
  X_out, _ = gen_fake_samples(gen_model, X_in, 0)
  
  ## rescale [-1,1] to [0,1]
  X_in = (X_in + 1) / 2.0
  X_out = (X_out + 1) / 2.0
  
  ## generate the plot fig to save.
  ## plt.subplot(nrows, ncols, index)
  n_rows = 2
  n_cols = n_samples
  
  ## original images
  for i in range(n_samples):
    plt.subplot(n_rows, n_cols, i+1)
    plt.axis('off')
    plt.imshow(X_in[i])
    
  ## translated images in the second row
  for i in range(n_samples):
    plt.subplot(n_rows, n_cols, n_samples+i+1)
    plt.axis('off')
    plt.imshow(X_out[i])
    
  ## save the plot
  filename = '%s_plot_e%02d_%04d.png' % (prefix, epoch, step+1)
  path = os.path.join(directory, filename)
  
  plt.savefig(path)
  plt.close()
  return

In [0]:
# define training
def train(n_epochs, n_batch_size, sample_interval, dis_A, dis_B, gen_AB, gen_BA, comb_AB, comb_BA, trainA, trainB):
  # get the patch size
  n_patch = dis_A.output_shape[1]
  
  # number of batches per epoch
  n_bat_per_epoch = int(len(trainA) / n_batch_size)
  
  # total number of steps to go thru
  #n_steps = n_bat_per_epoch * n_epochs
  
  # get the start time
  start_time = datetime.datetime.now()

  for epoch in range(1, n_epochs+1):
    #
    # go thru each step for training
    for i in range(n_bat_per_epoch):
      # get the real samples from both domains
      X_realA, y_realA = gen_real_samples(trainA, n_batch_size, n_patch)
      X_realB, y_realB = gen_real_samples(trainB, n_batch_size, n_patch)

      # get the translataed images for both domains
      X_fakeA, y_fakeA = gen_fake_samples(gen_BA, X_realB, n_patch)
      X_fakeB, y_fakeB = gen_fake_samples(gen_AB, X_realA, n_patch)

      # original paper, updates the pool for fake images. TODO.

      # Train generator_BA
      gA_loss = comb_BA.train_on_batch([X_realB, X_realA], [y_realA, X_realA, X_realB, X_realA])

      #--------------------
      # Train Discriminator A
      #--------------------
      dA_loss_real = dis_A.train_on_batch(X_realA, y_realA)
      dA_loss_fake = dis_A.train_on_batch(X_fakeA, y_fakeA)

      dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

      # Train generator_AB
      gB_loss = comb_AB.train_on_batch([X_realA, X_realB], [y_realB, X_realB, X_realA, X_realB])

      #--------------------
      # Train Discriminator B
      #--------------------    
      dB_loss_real = dis_B.train_on_batch(X_realB, y_realB)
      dB_loss_fake = dis_B.train_on_batch(X_fakeB, y_fakeB)
      dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

      # Total disciminator loss
      d_loss = 0.5 * np.add(dA_loss, dB_loss)

      # get the time
      elapsed_time = datetime.datetime.now() - start_time

      if ( i % sample_interval == 0):
        print ("[Epoch %d/%d] [Batch %d] [D loss: %f] [GA loss: %05f, GB loss: %05f ] time: %s " \
                                                                        % ( epoch, n_epochs,
                                                                            i,  
                                                                            d_loss,
                                                                            gA_loss[0],
                                                                            gB_loss[0],                                                
                                                                            elapsed_time))
        
        sample_images(epoch, i, gen_AB, trainA, 'AtoB', '/content/gen_images')
        sample_images(epoch, i, gen_BA, trainB, 'BtoA', '/content/gen_images')
        
    # every epoch takes more than one hour is GPU
    # so save models
    if ( epoch % 2 == 0):
      #save_models(epoch, '/content/drive/My Drive/App/CycleGAN/Faces/low_res/models', gen_AB, gen_BA, )
      save_models(epoch, '/content/models', gen_AB, gen_BA)
        

In [0]:
# then normalize the data
trainA = rescale_input(young_images)
trainB = rescale_input(old_images)

In [0]:
#trainA.shape
#trainA[1]
image_shape = trainA.shape[1:]

In [20]:
print(image_shape)

(128, 128, 3)


In [22]:
# define generators
# for resolution of 128, num resnet blocks is 6.
gen_AB = build_generator(image_shape, 6)
gen_BA = build_generator(image_shape, 6)

W0807 14:29:24.096668 140157669853056 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0807 14:29:24.141609 140157669853056 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0807 14:29:24.149805 140157669853056 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [23]:
# define discriminators
dis_A = build_discriminator(image_shape)
dis_B = build_discriminator(image_shape)

W0807 14:29:25.787989 140157669853056 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
# define the combined model
combined_model_AB = build_combined_model(image_shape, gen_AB, dis_B, gen_BA)
combined_model_BA = build_combined_model(image_shape, gen_BA, dis_A, gen_AB)

In [0]:
!rm -rf /content/gen_images
!rm -rf /content/models

In [0]:
!mkdir /content/gen_images
!mkdir /content/models

In [0]:
# perform training
n_epochs = 10
n_batch_size = 1
#n_patch_size = dis_A.output_shape[1]
sample_interval = 250

In [28]:
train(n_epochs, n_batch_size, sample_interval,
     dis_A, dis_B,
     gen_AB, gen_BA,
     combined_model_AB, combined_model_BA,
     trainA, 
     trainB)

W0807 14:29:37.553123 140157669853056 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:2741: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0807 14:29:37.558047 140157669853056 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Disc

[Epoch 1/10] [Batch 0] [D loss: 4.442421] [GA loss: 19.608568, GB loss: 17.686743 ] time: 0:01:24.640852 
[Epoch 1/10] [Batch 250] [D loss: 0.376362] [GA loss: 6.358499, GB loss: 5.150681 ] time: 0:06:22.992762 
[Epoch 2/10] [Batch 0] [D loss: 0.146605] [GA loss: 4.880492, GB loss: 5.028635 ] time: 0:11:19.825113 
[Epoch 2/10] [Batch 250] [D loss: 0.116012] [GA loss: 4.750803, GB loss: 4.103230 ] time: 0:16:16.767087 
[Epoch 3/10] [Batch 0] [D loss: 0.130084] [GA loss: 6.664525, GB loss: 6.279373 ] time: 0:21:57.761432 
[Epoch 3/10] [Batch 250] [D loss: 0.168301] [GA loss: 4.367022, GB loss: 4.220310 ] time: 0:26:54.576038 
[Epoch 4/10] [Batch 0] [D loss: 0.096039] [GA loss: 4.567014, GB loss: 4.132310 ] time: 0:31:52.139467 
[Epoch 4/10] [Batch 250] [D loss: 0.064165] [GA loss: 5.668515, GB loss: 4.658167 ] time: 0:36:46.472208 
[Epoch 5/10] [Batch 0] [D loss: 0.080418] [GA loss: 3.785238, GB loss: 3.654710 ] time: 0:41:40.436192 
[Epoch 5/10] [Batch 250] [D loss: 0.037025] [GA loss: 

In [29]:
!cp /content/gen_images/AtoB_plot_e07_0601.png '/content/drive/My Drive/App/CycleGAN/Faces/gen_images'
!cp /content/gen_images/AtoB_plot_e07_0401.png '/content/drive/My Drive/App/CycleGAN/Faces/gen_images'
!cp /content/gen_images/BtoA_plot_e07_0601.png '/content/drive/My Drive/App/CycleGAN/Faces/gen_images'
!cp /content/gen_images/BtoA_plot_e07_0401.png '/content/drive/My Drive/App/CycleGAN/Faces/gen_images'


cp: cannot stat '/content/gen_images/AtoB_plot_e07_0601.png': No such file or directory
cp: cannot stat '/content/gen_images/AtoB_plot_e07_0401.png': No such file or directory
cp: cannot stat '/content/gen_images/BtoA_plot_e07_0601.png': No such file or directory
cp: cannot stat '/content/gen_images/BtoA_plot_e07_0401.png': No such file or directory


In [0]:
!mv /content/gen_images /content/gen_images_run1
!mv /content/models /content/modles_run1

In [0]:
!mkdir /content/gen_images
!mkdir /content/models

## Train further more

In [32]:
# perform training
n_epochs = 50
train(n_epochs, n_batch_size, sample_interval,
     dis_A, dis_B,
     gen_AB, gen_BA,
     combined_model_AB, combined_model_BA,
     trainA, 
     trainB)

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[Epoch 1/50] [Batch 0] [D loss: 0.095882] [GA loss: 3.057039, GB loss: 2.628413 ] time: 0:00:01.326158 
[Epoch 1/50] [Batch 250] [D loss: 0.094835] [GA loss: 3.709941, GB loss: 3.152402 ] time: 0:04:55.177137 
[Epoch 2/50] [Batch 0] [D loss: 0.053088] [GA loss: 5.379539, GB loss: 6.167553 ] time: 0:09:48.226274 
[Epoch 2/50] [Batch 250] [D loss: 0.047193] [GA loss: 4.057099, GB loss: 3.546183 ] time: 0:14:42.471570 
[Epoch 3/50] [Batch 0] [D loss: 0.083339] [GA loss: 2.768277, GB loss: 2.671884 ] time: 0:19:39.485605 
[Epoch 3/50] [Batch 250] [D loss: 0.121297] [GA loss: 4.727778, GB loss: 3.830541 ] time: 0:24:36.232405 
[Epoch 4/50] [Batch 0] [D loss: 0.044829] [GA loss: 4.191004, GB loss: 3.946934 ] time: 0:29:31.926103 
[Epoch 4/50] [Batch 250] [D loss: 0.081432] [GA loss: 4.358021, GB loss: 3.611142 ] time: 0:34:26.582400 
[Epoch 5/50] [Batch 0] [D loss: 0.023520] [GA loss: 3.770883, GB loss: 3.308651 ] time: 0:39:20.883068 
[Epoch 5/50] [Batch 250] [D loss: 0.068644] [GA loss: 2.

In [0]:
dis_A.save('/content/drive/My Drive/App/CycleGAN/Faces/low_res/models/dis_A')

In [0]:
dis_B.save('/content/drive/My Drive/App/CycleGAN/Faces/low_res/models/dis_B')

In [0]:
gen_AB.save('/content/drive/My Drive/App/CycleGAN/Faces/low_res/models/gen_AB')

In [0]:
gen_BA.save('/content/drive/My Drive/App/CycleGAN/Faces/low_res/models/gen_BA')

In [0]:
combined_model_AB.save('/content/drive/My Drive/App/CycleGAN/Faces/low_res/models/comb_AB')

In [0]:
combined_model_BA.save('/content/drive/My Drive/App/CycleGAN/Faces/low_res/models/comb_BA')

In [0]:
!cp '/content/gen_images/AtoB_plot_e49_0251.png' '/content/drive/My Drive/App/CycleGAN/Faces/low_res/gen_images'

In [0]:
!cp '/content/gen_images/BtoA_plot_e49_0251.png' '/content/drive/My Drive/App/CycleGAN/Faces/low_res/gen_images'

In [0]:
!cp '/content/models/gen_modelAB_e050.hdf5' '/content/drive/My Drive/App/CycleGAN/Faces/low_res/models/gen_modelAB_e050.hdf5'

In [0]:
!cp '/content/models/gen_modelBA_e050.hdf5' '/content/drive/My Drive/App/CycleGAN/Faces/low_res/models/gen_modelBA_e050.hdf5'

In [0]:
!mv '/content/gen_images' '/content/gen_images_run2'
!mv '/content/models' '/content/models_run2'

In [0]:
!mkdir '/content/gen_images'
!mkdir '/content/models'

## Load the trained models

In [0]:
!cp -r '/content/drive/My Drive/App/CycleGAN/Faces/low_res/models' /content



In [0]:
#!cp '/content/drive/My Drive/App/CycleGAN/Faces/low_res/models/gen_BA' /conent
#!cp '/content/drive/My Drive/App/CycleGAN/Faces/low_res/models/dis_A' /conent
#!cp '/content/drive/My Drive/App/CycleGAN/Faces/low_res/models/dis_B' /conent
#!cp '/content/drive/My Drive/App/CycleGAN/Faces/low_res/models/comb_AB' /conent
#!cp '/content/drive/My Drive/App/CycleGAN/Faces/low_res/models/comb_BA' /conent

In [0]:
!mv /content/models/dis_A /content/models/dis_A.hdf5

In [0]:
from keras.models import load_model
# from keras_contrib
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization


In [38]:
cust = {'InstanceNormalization':InstanceNormalization}
dis_A = load_model('/content/models/dis_A.hdf5', cust)

W0810 05:39:36.870250 139721337706368 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0810 05:39:37.175507 139721337706368 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0810 05:39:37.178967 139721337706368 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0810 05:39:37.181187 139721337706368 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:186: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0810 05:39:40.235575 139721337706368 deprecation_w

In [0]:
dis_B   = load_model('/content/models/dis_B',cust)

In [42]:
gen_AB  = load_model('/content/models/gen_AB',cust)
gen_BA  = load_model('/content/models/gen_BA',cust)

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [43]:
comb_AB = load_model('/content/models/comb_AB',cust)
comb_BA = load_model('/content/models/comb_BA',cust)

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:327: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:327: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


## Train 10 more (61 thru 70)

In [0]:
!mkdir /content/gen_images
#!mkdir /content/models

In [0]:
!mv /content/models /content/old_models

In [0]:
!mkdir /content/models

In [51]:
# perform training
n_epochs = 10
train(n_epochs, n_batch_size, sample_interval,
     dis_A, dis_B,
     gen_AB, gen_BA,
     comb_AB, comb_BA,
     trainA, 
     trainB)

[Epoch 1/10] [Batch 0] [D loss: 0.019375] [GA loss: 3.152634, GB loss: 3.122714 ] time: 0:01:18.218249 
[Epoch 1/10] [Batch 250] [D loss: 0.012319] [GA loss: 1.970519, GB loss: 1.686912 ] time: 0:07:55.381372 
[Epoch 2/10] [Batch 0] [D loss: 0.027718] [GA loss: 1.643641, GB loss: 1.305251 ] time: 0:14:31.218902 
[Epoch 2/10] [Batch 250] [D loss: 0.010343] [GA loss: 1.457999, GB loss: 1.535118 ] time: 0:21:06.879577 
[Epoch 3/10] [Batch 0] [D loss: 0.012809] [GA loss: 1.873905, GB loss: 2.364418 ] time: 0:29:22.393632 
[Epoch 3/10] [Batch 250] [D loss: 0.020429] [GA loss: 2.051445, GB loss: 2.400681 ] time: 0:35:58.125933 
[Epoch 4/10] [Batch 0] [D loss: 0.007793] [GA loss: 2.697351, GB loss: 2.392731 ] time: 0:42:33.593235 
[Epoch 4/10] [Batch 250] [D loss: 0.005521] [GA loss: 1.299808, GB loss: 1.873229 ] time: 0:49:09.454148 
[Epoch 5/10] [Batch 0] [D loss: 0.005942] [GA loss: 1.285210, GB loss: 1.530530 ] time: 0:55:45.629594 
[Epoch 5/10] [Batch 250] [D loss: 0.025968] [GA loss: 3.

In [0]:
!cp '/content/models/gen_modelAB_e010.hdf5' '/content/drive/My Drive/App/CycleGAN/Faces/low_res/models/gen_AB_e70.hdf5'

In [0]:
!cp '/content/models/gen_modelBA_e010.hdf5' '/content/drive/My Drive/App/CycleGAN/Faces/low_res/models/gen_BA_e70.hdf5'

In [0]:
!cp -r '/content/gen_images' '/content/drive/My Drive/App/CycleGAN/Faces/low_res/gen_images_e70'

In [0]:
dis_A.save('/content/drive/My Drive/App/CycleGAN/Faces/low_res/models_e70/dis_A')
dis_B.save('/content/drive/My Drive/App/CycleGAN/Faces/low_res/models_e70/dis_B')
gen_AB.save('/content/drive/My Drive/App/CycleGAN/Faces/low_res/models_e70/gen_AB')
gen_BA.save('/content/drive/My Drive/App/CycleGAN/Faces/low_res/models_e70/gen_BA')


In [0]:
comb_AB.save('/content/drive/My Drive/App/CycleGAN/Faces/low_res/models_e70/comb_AB')
comb_BA.save('/content/drive/My Drive/App/CycleGAN/Faces/low_res/models_e70/comb_BA')

In [0]:
comb_AB.save('/content/comb_AB.hdf5')

In [0]:
comb_BA.save('/content/comb_BA.hdf5')

In [0]:
!cp '/content/comb_AB.hdf5' '/content/drive/My Drive/App/CycleGAN/Faces/low_res/models_e70/comb_AB.hdf5'

In [0]:
!cp '/content/comb_BA.hdf5' '/content/drive/My Drive/App/CycleGAN/Faces/low_res/models_e70/comb_BA.hdf5'

In [0]:
!mv /content/gen_images /content/gen_images_e70
!mv /content/models /content/models_e70

In [0]:
!mkdir /content/models
!mkdir /content/gen_images

## Train some more (Epochs 70 thru 90)

In [70]:
# perform training
n_epochs = 20
train(n_epochs, n_batch_size, sample_interval,
     dis_A, dis_B,
     gen_AB, gen_BA,
     comb_AB, comb_BA,
     trainA, 
     trainB)

[Epoch 1/20] [Batch 0] [D loss: 0.004783] [GA loss: 1.663634, GB loss: 1.603557 ] time: 0:00:01.732597 
[Epoch 1/20] [Batch 250] [D loss: 0.003340] [GA loss: 1.365775, GB loss: 1.464820 ] time: 0:06:36.440047 
[Epoch 2/20] [Batch 0] [D loss: 0.007187] [GA loss: 1.259685, GB loss: 1.529884 ] time: 0:13:10.841447 
[Epoch 2/20] [Batch 250] [D loss: 0.002996] [GA loss: 2.636429, GB loss: 2.261585 ] time: 0:19:45.176121 
[Epoch 3/20] [Batch 0] [D loss: 0.002116] [GA loss: 1.667826, GB loss: 1.766800 ] time: 0:26:19.543273 
[Epoch 3/20] [Batch 250] [D loss: 0.003143] [GA loss: 1.855464, GB loss: 1.697687 ] time: 0:32:53.083043 
[Epoch 4/20] [Batch 0] [D loss: 0.008835] [GA loss: 1.517345, GB loss: 1.985160 ] time: 0:39:26.883675 
[Epoch 4/20] [Batch 250] [D loss: 0.003478] [GA loss: 1.172623, GB loss: 1.239328 ] time: 0:46:01.101361 
[Epoch 5/20] [Batch 0] [D loss: 0.002532] [GA loss: 2.726803, GB loss: 2.176687 ] time: 0:52:35.634328 
[Epoch 5/20] [Batch 250] [D loss: 0.002819] [GA loss: 1.

In [0]:
dis_A.save('/content/drive/My Drive/App/CycleGAN/Faces/low_res/models_e90/dis_A')
dis_B.save('/content/drive/My Drive/App/CycleGAN/Faces/low_res/models_e90/dis_B')
gen_AB.save('/content/drive/My Drive/App/CycleGAN/Faces/low_res/models_e90/gen_AB')
gen_BA.save('/content/drive/My Drive/App/CycleGAN/Faces/low_res/models_e90/gen_BA')
comb_AB.save('/content/drive/My Drive/App/CycleGAN/Faces/low_res/models_e90/comb_AB')
comb_BA.save('/content/drive/My Drive/App/CycleGAN/Faces/low_res/models_e90/comb_BA')

In [0]:
!cp -r /content/gen_images '/content/drive/My Drive/App/CycleGAN/Faces/low_res/gen_images_e90'

In [0]:
!mv gen_images gen_images_90
!mv models models_90

In [0]:
!mkdir gen_images
!mkdir models

## Train more (epochs 90 thru 100)

In [77]:
# perform training
n_epochs = 10
train(n_epochs, n_batch_size, sample_interval,
     dis_A, dis_B,
     gen_AB, gen_BA,
     comb_AB, comb_BA,
     trainA, 
     trainB)

[Epoch 1/10] [Batch 0] [D loss: 0.000805] [GA loss: 1.571090, GB loss: 1.327103 ] time: 0:00:01.689662 
[Epoch 1/10] [Batch 250] [D loss: 0.002237] [GA loss: 1.301857, GB loss: 1.335426 ] time: 0:06:27.990259 
[Epoch 2/10] [Batch 0] [D loss: 0.001656] [GA loss: 1.217103, GB loss: 1.522710 ] time: 0:12:53.982364 
[Epoch 2/10] [Batch 250] [D loss: 0.000936] [GA loss: 1.378377, GB loss: 1.250276 ] time: 0:19:21.774303 
[Epoch 3/10] [Batch 0] [D loss: 0.001940] [GA loss: 1.627454, GB loss: 1.874849 ] time: 0:25:50.720505 
[Epoch 3/10] [Batch 250] [D loss: 0.002066] [GA loss: 1.652478, GB loss: 1.570552 ] time: 0:32:17.013869 
[Epoch 4/10] [Batch 0] [D loss: 0.002477] [GA loss: 1.444516, GB loss: 1.541596 ] time: 0:38:44.801013 
[Epoch 4/10] [Batch 250] [D loss: 0.001667] [GA loss: 2.107458, GB loss: 1.925315 ] time: 0:45:12.107604 
[Epoch 5/10] [Batch 0] [D loss: 0.001351] [GA loss: 1.013837, GB loss: 1.152637 ] time: 0:51:41.040292 
[Epoch 5/10] [Batch 250] [D loss: 0.000536] [GA loss: 1.

In [0]:
dis_A.save('/content/drive/My Drive/App/CycleGAN/Faces/low_res/models_e100/dis_A')
dis_B.save('/content/drive/My Drive/App/CycleGAN/Faces/low_res/models_e100/dis_B')
gen_AB.save('/content/drive/My Drive/App/CycleGAN/Faces/low_res/models_e100/gen_AB')
gen_BA.save('/content/drive/My Drive/App/CycleGAN/Faces/low_res/models_e100/gen_BA')
comb_AB.save('/content/drive/My Drive/App/CycleGAN/Faces/low_res/models_e100/comb_AB')
comb_BA.save('/content/drive/My Drive/App/CycleGAN/Faces/low_res/models_e100/comb_BA')

In [0]:
!cp -r /content/gen_images '/content/drive/My Drive/App/CycleGAN/Faces/low_res/gen_images_e100'